In [1]:
import torch
from torch import nn 
from torch.utils.data import Dataset, DataLoader
import os
import pandas as pd

In [ ]:
class PipesDataset(Dataset):
    def __init__(self, path: str):
        """
        Args:
            path (str): Path to the dataset file
        
        The dataset file should be a CSV file with two columns:
        - The first column should contain the state of the game, represented by a string of digits
            - Each digit represents whether an opening is open or closed, with 0 being closed and 1 being open
            - The first digit represents top-opening on the first pipe, second represents right-opening on the first pipe, and so on
        - The second column should contain the action taken
            - The action should be an integer, representing the index of the pipe to turn clockwise
        """
        curr_dir = os.getcwd()
        data_path = os.path.join(curr_dir, path)
        self.df = pd.read_csv(data_path)
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        state = self.df.iloc[idx, 0]
        state_int_list = [int(x) for x in state]
        state_tensor = torch.tensor(state_int_list)

        action = self.df.iloc[idx, 1]
        action_tensor = torch.tensor(action)

        return state_tensor, action_tensor


train_pipes = DataLoader(PipesDataset("data/train.csv"), batch_size=64, shuffle=True)
test_pipes = DataLoader(PipesDataset("data/test.csv"), batch_size=64, shuffle=True)
train_features, train_labels = next(iter(train_pipes))
test_features, test_labels = next(iter(test_pipes))

In [3]:
class PipePredictor(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_classes),
        )
        
    def forward(self, x):
        return self.network(x)

In [ ]:
n = 4
model = PipePredictor(input_size=4*(n**2), hidden_size=64, num_classes=n**2)

learning_rate = 1e-3
batch_size = 64
epochs = 5

# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device).float(), y.to(device)
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_pipes, model, loss_fn, optimizer)
    test_loop(test_pipes, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float